In [1]:
from google.colab import drive, files
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


#Kaggle env

In [2]:
#!pip install -q kaggle

#kaggle key
!mkdir ~/.kaggle
!cp /content/drive/My\ Drive/kaggle.json ~/.kaggle
!ls ~/.kaggle


kaggle.json


#Download data

In [3]:
!kaggle competitions download -c title-generation

 78% 5.00M/6.39M [00:00<00:00, 22.7MB/s]
100% 6.39M/6.39M [00:00<00:00, 25.4MB/s]
  0% 0.00/905k [00:00<?, ?B/s]
100% 905k/905k [00:00<00:00, 59.5MB/s]
  0% 0.00/778k [00:00<?, ?B/s]
100% 778k/778k [00:00<00:00, 106MB/s]
 94% 41.0M/43.7M [00:02<00:00, 12.1MB/s]
100% 43.7M/43.7M [00:02<00:00, 18.2MB/s]


In [4]:
!mkdir data
!unzip sample_submission.csv.zip -d data
!unzip vocs.pkl.zip -d data
!unzip train.csv.zip -d data
!mv test.csv data

!ls data


Archive:  sample_submission.csv.zip
  inflating: data/sample_submission.csv  
Archive:  vocs.pkl.zip
  inflating: data/vocs.pkl           
Archive:  train.csv.zip
  inflating: data/train.csv          
sample_submission.csv  test.csv  train.csv  vocs.pkl


In [0]:
!cat data/test.csv 

In [5]:
import pandas as pd

train = pd.read_csv('data/train.csv', encoding='utf8')
test = pd.read_csv('data/test.csv', encoding='utf8')

from sklearn.model_selection import train_test_split

train, val = train_test_split(train, test_size=0.2)

print(train.shape,val.shape,test.shape)

(108000, 2) (27000, 2) (1000, 1)


In [0]:
train.head()

,abstract,title
0,we consider the problem of utility maximizatio...,on optimal investment with processes of long o...
1,in this paper we provide an explicit formula f...,boolean complexes for ferrers graphs
2,"kinesin-5, also known as eg5 in vertebrates is...",relative velocity of sliding of microtubules b...
3,we discuss the transition paths in a coupled b...,bifurcation of transition paths induced by cou...
4,two types of room temperature detectors of ter...,all-electric detectors of the polarization sta...


In [0]:
test.head()

,abstract
0,Most sequence transformation models use recurr...
1,The doc2vec approach was introduced as an exte...
2,LSTM models can vary greatly depending on sequ...
3,A joint learning process of alignment and tran...
4,Current unsupervised image-to-image translatio...


In [13]:
train.title

133191            absence of magnetic ordering in niga_2s_4
106691    the almost sure behavior of certain spatial re...
17970     therapeutic hypothermia: quantification of the...
7132      bayesian dose-response analysis for epidemiolo...
24508     lifetime measurements of triaxial strongly def...
                                ...                        
125812    dimension reduction of clustering results in b...
50189     demystifying fixed k-nearest neighbor informat...
130735    mathematical analysis of long tail economy usi...
60979     characteristics of transposable element exoniz...
2876      multiplierless 16-point dct approximation for ...
Name: title, Length: 108000, dtype: object

#BertSum prepare


#BERT test

In [0]:
class InputExample():
    """A single training/test example for simple sequence classification."""

    def __init__(self, guid, text_a, text_b=None):
        """Constructs a InputExample.
        Args:
            guid: Unique id for the example.
            text_a: string. The untokenized text of the first sequence.
                For single sequence tasks, only this sequence must be specified.
            text_b: (Optional) string. The untokenized text of the second
                sequence. Only must be specified for sequence pair tasks.
            label: (Optional) string. The label of the example. This should be
                specified for train and dev examples, but not for test examples.
        """
        self.guid = guid
        self.src_txt = text_a
        self.tgt_txt = text_b

def _create_examples(self, src_lines,tgt_lines,set_type):
    examples = [] 
    for i,data in enumerate(zip(src_lines,tgt_lines)):
        guid = "%s-%s" % (set_type, i)
        if len(data[0])==0 or len(data[1])==0:
          continue
        src_lines = tokenization.convert_to_unicode(data[0][0])
        tgt_lines = tokenization.convert_to_unicode(data[1][0])
        examples.append(InputExample(guid=guid, text_a=src_lines,
                                  text_b=tgt_lines))
    return examples

In [0]:
# Load pre-trained model tokenizer (vocabulary)
# modelpath = "bert-base-uncased"
# tokenizer = BertTokenizer.from_pretrained(modelpath)

set_type = 'train'
examples = [] 
for row in train[:3].iterrows():
    i,(src_lines,tgt_lines) = row
    guid = "%s-%s" % (set_type, i)
    if len(src_lines)==0 or len(tgt_lines)==0:
      continue
    src_lines = tokenizer.tokenize(src_lines)
    tgt_lines = tokenizer.tokenize(tgt_lines)
    examples.append(InputExample(guid=guid, text_a=src_lines,
                              text_b=tgt_lines))


In [0]:
def _format_to_bert(params):
    json_file, args, save_file = params
    if (os.path.exists(save_file)):
        logger.info('Ignore %s' % save_file)
        return

    bert = BertData(args)

    logger.info('Processing %s' % json_file)
    jobs = json.load(open(json_file))
    datasets = []
    for d in jobs:
        source, tgt = d['src'], d['tgt']
        if (args.oracle_mode == 'greedy'):
            oracle_ids = greedy_selection(source, tgt, 3)
        elif (args.oracle_mode == 'combination'):
            oracle_ids = combination_selection(source, tgt, 3)
        b_data = bert.preprocess(source, tgt, oracle_ids)
        if (b_data is None):
            continue
        indexed_tokens, labels, segments_ids, cls_ids, src_txt, tgt_txt = b_data
        b_data_dict = {"src": indexed_tokens, "labels": labels, "segs": segments_ids, 'clss': cls_ids,
                       'src_txt': src_txt, "tgt_txt": tgt_txt}
        datasets.append(b_data_dict)
    logger.info('Saving to %s' % save_file)
    torch.save(datasets, save_file)
    datasets = []
    gc.collect()

In [0]:

#inpf = convert_single_example(1,examples[0],75,75,tokenizer)
inpf.src_segment_ids

In [0]:
class InputFeatures():
    """A single set of features of data."""

    def __init__(self, src_input_ids,src_input_mask,src_segment_ids,tgt_input_ids,tgt_input_mask,tgt_labels):
        self.src_input_ids = src_input_ids
        self.src_input_mask = src_input_mask
        self.src_segment_ids = src_segment_ids
        self.tgt_input_ids = tgt_input_ids
        self.tgt_input_mask = tgt_input_mask 
        self.tgt_labels = tgt_labels
        
def convert_single_example(ex_index, example, max_seq_length_src,max_seq_length_tgt,
                           tokenizer):
    """Converts a single `InputExample` into a single `InputFeatures`."""
    """
    label_map = {}
    for (i, label) in enumerate(label_list):
        label_map[label] = i
    """
    # tokens_a = tokenizer.tokenize(example.src_txt)
    # tokens_b = tokenizer.tokenize(example.tgt_txt)

    tokens_a = example.src_txt
    tokens_b = example.tgt_txt


    # Modifies `tokens_a` and `tokens_b` in place so that the total
    # length is less than the specified length.
    # Account for [CLS], [SEP], [SEP] with "- 3"
    if len(tokens_a) > max_seq_length_src - 2:
            tokens_a = tokens_a[0:(max_seq_length_src - 2)]
    
    if len(tokens_b) > max_seq_length_tgt - 2:
            tokens_b = tokens_b[0:(max_seq_length_tgt - 2)]

    
    tokens_src = []
    segment_ids_src = []
    tokens_src.append("[CLS]")
    segment_ids_src.append(0)
    for token in tokens_a:
        tokens_src.append(token)
        segment_ids_src.append(0)
    tokens_src.append("[SEP]")
    segment_ids_src.append(0)
  

    tokens_tgt = []
    segment_ids_tgt = []
    tokens_tgt.append("[CLS]")
    #segment_ids_tgt.append(0)
    for token in tokens_b:
        tokens_tgt.append(token)
        #segment_ids_tgt.append(0)
    tokens_tgt.append("[SEP]")
    #segment_ids_tgt.append(0)

    input_ids_src = tokenizer.convert_tokens_to_ids(tokens_src)
   
    

    input_ids_tgt = tokenizer.convert_tokens_to_ids(tokens_tgt)
    
    #Adding begiining and end token
    input_ids_tgt = input_ids_tgt[:-1] 
    
    input_mask_src = [1] * len(input_ids_src)


    input_mask_tgt = [1] * len(input_ids_tgt)
    
    labels_tgt = input_ids_tgt[1:]
    
    
    labels_tgt.append(0)
    
    #print(len(input_ids_tgt))
    #print(len(input_mask_tgt))
    #print(len(labels_tgt))
    #print(len(segment_ids_tgt))
    
    while len(input_ids_src) < max_seq_length_src:
        input_ids_src.append(0)
        input_mask_src.append(0)
        segment_ids_src.append(0)

    while len(input_ids_tgt) < max_seq_length_tgt:
        input_ids_tgt.append(0)
        input_mask_tgt.append(0)
        segment_ids_tgt.append(0)
        labels_tgt.append(0)

    feature = InputFeatures( src_input_ids=input_ids_src,src_input_mask=input_mask_src,src_segment_ids=segment_ids_src,
        tgt_input_ids=input_ids_tgt,tgt_input_mask=input_mask_tgt,tgt_labels=labels_tgt)

    
    return feature

In [38]:
import torch
from pytorch_transformers import BertTokenizer, BertModel, BertForMaskedLM

# Load pre-trained model tokenizer (vocabulary)
modelpath = "bert-base-uncased"
tokenizer = BertTokenizer.from_pretrained(modelpath)

text = "dummy. although he had already eaten a large meal, he was still very hungry."
target = "hungry"
tokenized_text = tokenizer.tokenize(text)

# Mask a token that we will try to predict back with `BertForMaskedLM`
masked_index = tokenized_text.index(target)
tokenized_text[masked_index] = '[MASK]'

# Convert token to vocabulary indices
indexed_tokens = tokenizer.convert_tokens_to_ids(tokenized_text)
# Define sentence A and B indices associated to 1st and 2nd sentences (see paper)
segments_ids = [1] * len(tokenized_text)
# this is for the dummy first sentence. 
segments_ids[0] = 0
segments_ids[1] = 0

# Convert inputs to PyTorch tensors
tokens_tensor = torch.tensor([indexed_tokens])
segments_tensors = torch.tensor([segments_ids])
# Load pre-trained model (weights)
model = BertForMaskedLM.from_pretrained(modelpath)
model.eval()

# Predict all tokens
predictions = model(tokens_tensor, segments_tensors)
predicted_index = torch.argmax(predictions[0][0, masked_index]).item()
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])

print("Original:", text)
print("Masked:", " ".join(tokenized_text))

print("Predicted token:", predicted_token)
print("Other options:")
# just curious about what the next few options look like.
for i in range(10):
    predictions[0][0,masked_index,predicted_index] = -11100000
    predicted_index = torch.argmax(predictions[0][0, masked_index]).item()
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])
    print(predicted_token)

Original: dummy. although he had already eaten a large meal, he was still very hungry.
Masked: dummy . although he had already eaten a large meal , he was still very [MASK] .
Predicted token: ['hungry']
Other options:
['strong']
['tired']
['weak']
['angry']
['concerned']
['alert']
['nervous']
['pale']
['.']
['excited']


In [36]:
#predicted_index = torch.argmax(predictions[0, masked_index]).item()
predictions[0][0,15] #, masked_index]

tensor([-5.0937, -5.0305, -5.1461,  ..., -3.5266, -4.0072, -3.8648],
       grad_fn=<SelectBackward>)

#Summarizer

In [0]:
!pip install bert-extractive-summarizer

In [19]:
from summarizer import Summarizer

model = Summarizer()

model(train.abstract[0])

100%|██████████| 40155833/40155833 [00:02<00:00, 18096237.37B/s]


'we consider the problem of utility maximization for investors with power utility functions. in such models there is no hope to get a formula for the achievable maximal utility.'

In [21]:
train.title[0]

'on optimal investment with processes of long or negative memory'

#BERT

In [0]:
!pip install pytorch-transformers

In [7]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from pytorch_transformers import BertTokenizer, BertConfig
from pytorch_transformers import AdamW, BertForSequenceClassification
from tqdm import tqdm, trange
import pandas as pd
import io
import numpy as np
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

Using TensorFlow backend.


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
if device == 'cpu':
    print('cpu')
else:
    n_gpu = torch.cuda.device_count()
    print(torch.cuda.get_device_name(0))

Tesla T4


In [9]:
# vocab_file = 'data/vocs.pkl'
# tokenizer = BertTokenizer(vocab_file=vocab_file, do_lower_case=True) 
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

100%|██████████| 231508/231508 [00:00<00:00, 901877.04B/s]


In [0]:
tokenized_texts = [tokenizer.tokenize(sent) for sent in train.abstract.values]

In [16]:
print(tokenized_texts[0])

['triangular', '-', 'layered', 'ni', '##ga', '##2', '##s', '##4', ',', 'contrary', 'to', 'intuitive', 'expectation', ',', 'does', 'not', 'form', 'a', 'non', '##coll', '##ine', '##ar', 'anti', '##fer', '##rom', '##agne', '##tic', 'structure', ',', 'as', 'do', 'iso', '##ele', '##ct', '##ron', '##ic', 'na', '##cr', '##o', '##2', 'and', 'li', '##cr', '##o', '##2', '.', 'instead', ',', 'the', 'local', 'magnetic', 'moments', 'remain', 'disorder', '##ed', 'down', 'to', 'the', 'lowest', 'measured', 'temperature', '.', 'to', 'get', 'more', 'insight', 'into', 'this', 'phenomenon', ',', 'we', 'have', 'performed', 'first', 'principles', 'calculations', 'of', 'the', 'first', ',', 'second', 'end', 'third', 'neighbors', 'exchange', 'interactions', ',', 'and', 'found', 'that', 'the', 'second', 'neighbor', 'exchange', 'is', 'ne', '##gli', '##gible', ',', 'while', 'the', 'first', 'and', 'the', 'third', 'neighbor', 'exchanges', 'are', 'comparable', 'and', 'anti', '##fer', '##rom', '##agne', '##tic', '.',

In [0]:
MAX_LEN = 1500

input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in tokenized_texts],
                          maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")

In [0]:
attention_masks = [[float(i>0) for i in ii] for ii in input_ids]

In [0]:
attention_masks[0]

#Seq2Seq

In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.data import Field, BucketIterator

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

import spacy

import random
import math
import time

SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

spacy_en = spacy.load('en')